# Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

#### 插入NaN，用 np.nan
#### 判断一个表各个位置是否是NaN，用 .isnull()
#### 也可以用索引=None，在特定位置插入 NaN

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [3]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

#### df.dropna() 可以把NaN的行去除，相当于 df[df.notnull()]

In [3]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [5]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

#### df 的.dropna() 会把所有的包含NaN的行都删除
#### .dropna() 可以使用不同参数如how='all'：全为NaN的行才删除

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [8]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [9]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


#### data[4] = NA 添加一列全部为NaN的列
#### .dropna(axis=1, how='all')，删除全为NaN的列（axis=1)

In [10]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [11]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2)

,0,1,2
2,0.331286,NaN,0.069877
3,0.246674,NaN,1.004812
4,1.327195,-0.919262,-1.549106
5,0.022185,0.758363,-0.660524
6,0.862580,-0.010032,0.050009


### Filling In Missing Data

#### 指定NA值的补充用df.fillna 函数
#### 1. 直接补充特定值    2. 用字典指定不同列用什么值    3. 如果不加inplace=True，就不用重新赋值，更改会自动被保存
#### 4. method='ffill' 可以用旁边的值去补齐   5. 也可以用 data.mean(）使用均值补齐

In [20]:
df.fillna(0)

,0,1,2
0,-1.541996,0.000000,0.000000
1,0.286350,0.000000,0.000000
2,0.331286,0.000000,0.069877
3,0.246674,0.000000,1.004812
4,1.327195,-0.919262,-1.549106
5,0.022185,0.758363,-0.660524
6,0.862580,-0.010032,0.050009


In [21]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-1.541996,0.500000,0.000000
1,0.286350,0.500000,0.000000
2,0.331286,0.500000,0.069877
3,0.246674,0.500000,1.004812
4,1.327195,-0.919262,-1.549106
5,0.022185,0.758363,-0.660524
6,0.862580,-0.010032,0.050009


In [22]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-1.541996,0.000000,0.000000
1,0.286350,0.000000,0.000000
2,0.331286,0.000000,0.069877
3,0.246674,0.000000,1.004812
4,1.327195,-0.919262,-1.549106
5,0.022185,0.758363,-0.660524
6,0.862580,-0.010032,0.050009


In [4]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

,0,1,2
0,1.669025,-0.438570,-0.539741
1,0.476985,3.248944,-1.021228
2,-0.577087,3.248944,0.302614
3,0.523772,3.248944,1.343810
4,-0.713544,NaN,1.343810
5,-1.860761,NaN,1.343810


In [5]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation

### Removing Duplicates

#### df.duplicated() 可以判断df是否有重复行 （返回是一列bool值）
#### df.drop_duplicates()  去除重复的行
#### data.drop_duplicates(['列名']) 去除某列出现重复的行，可以用 list 形式
#### 这个函数，包含的参数 keep='last' 时，删除靠前的重复行（默认删除后出现的行）

In [6]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [7]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [8]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [9]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [10]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [11]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


#### 当存在一个额外的表时，如何实现某列的映射？例如上表'bacon'和'pulled pork'都属于'pig'，'pastrami'和'corned beef'都属于'cow'
#### 准备工作：用一个字典dicty表示这种映射，把需要关联的列x的数据清洗好，如规范大小写等
#### df['new col'] = x.map(dicty)

In [13]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [14]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [15]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


#### 也可以使用lambda函数一行写完

In [16]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

#### 替换表中的值：df.replace(x, y)
#### 1. x 和 y 都为值 (np.nan也可以）
#### 2. x可以为列表
#### 3. x 和 y 可以是长度相同的列表
#### 4. 也可以是字典的一一对应形式

In [17]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [18]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [19]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [20]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [21]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [23]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


 #### 更改行名(index) ：
 #### 1. 用 data.index.map(lambda表达式)
 #### 2. 用几种固定模式 data.rename(index=str.title, columns=str.upper)，str.title表示首字母大写其他小写，str.upper表示全大写
 #### 3. data.rename这种形式可以把 index 或 columns的参数用字典表示 实现一一映射
 #### 4. 默认不自动存储，若需要可以使用 inplace=True

In [24]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [25]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [26]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [27]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [28]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [32]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

#### 对数据（特别是连续值）进行分组
### 1. 指定断点为一个list 使用 pd.cut 函数，默认左开右闭，参数 right=False 可以改成左闭右开
#### 2. cats.codes是类别序号0123，cats.categories是区间，pd.value_counts(cats)查看各组样本数目
#### 3. labels=列表 给各组数据命名
### 4. pd.cut(data, 4, precision=2) 是另一种分组法，将数据平均分成4份，每组样本数相同。
### 5. pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) 是取分为数法

In [33]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [40]:
print(cats.codes, '\n **************************** \n',
cats.categories, '\n *************************** \n ',
pd.value_counts(cats))

[0 0 0 1 0 0 2 1 3 2 2 1] 
 **************************** 
 IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]') 
 *************************** 
  (18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64


In [41]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [42]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [43]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.34, 0.55], (0.12, 0.34], (0.76, 0.97], (0.76, 0.97], (0.55, 0.76], ..., (0.55, 0.76], (0.76, 0.97], (0.55, 0.76], (0.76, 0.97], (0.55, 0.76]]
Length: 20
Categories (4, interval[float64]): [(0.12, 0.34] < (0.34, 0.55] < (0.55, 0.76] < (0.76, 0.97]]

In [44]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats
pd.value_counts(cats)

(-2.9499999999999997, -0.684]    250
(-0.684, -0.0231]                250
(-0.0231, 0.624]                 250
(0.624, 3.928]                   250
dtype: int64

In [45]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0231, 1.287], (-0.0231, 1.287], (1.287, 3.928], (-1.187, -0.0231], (-2.9499999999999997, -1.187], ..., (-2.9499999999999997, -1.187], (-1.187, -0.0231], (-0.0231, 1.287], (-1.187, -0.0231], (-0.0231, 1.287]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0231] < (-0.0231, 1.287] < (1.287, 3.928]]

### Detecting and Filtering Outliers

#### data.describe()快速展示一个df各列的均值标准差分位数等基本信息

In [46]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.051869,0.049129,0.023669,0.000935
std,0.998159,0.996927,1.007615,0.993223
min,-3.428254,-3.645860,-3.184377,-3.745356
25%,-0.743886,-0.599807,-0.617533,-0.682799
50%,-0.091364,0.047101,-0.016127,-0.020427
75%,0.623331,0.756646,0.690847,0.700701
max,3.366626,2.653656,3.525865,2.735527


In [47]:
col = data[2]
col[np.abs(col) > 3]

61     3.260383
236   -3.056990
323   -3.184377
783    3.525865
Name: 2, dtype: float64

#### 要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用any方法

In [48]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
42,-2.315555,0.457246,-0.025907,-3.399312
61,0.050188,1.951312,3.260383,0.963301
137,0.146326,0.508391,-0.196713,-3.745356
236,-0.293333,-0.242459,-3.056990,1.918403
260,-3.428254,-0.296336,-0.439938,-0.867165
323,0.275144,1.179227,-3.184377,1.369891
545,-0.362528,-3.548824,1.553205,-2.186301
637,3.366626,-2.372214,0.851010,1.332846
783,-0.658090,-0.207434,3.525865,0.283070
804,0.599947,-3.645860,0.255475,-0.549574


In [49]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.051807,0.050324,0.023124,0.002080
std,0.995608,0.992900,1.004370,0.989392
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.743886,-0.599807,-0.617533,-0.682799
50%,-0.091364,0.047101,-0.016127,-0.020427
75%,0.623331,0.756646,0.690847,0.700701
max,3.000000,2.653656,3.000000,2.735527


In [50]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,-1.0,1.0
1,-1.0,-1.0,1.0,-1.0
2,1.0,1.0,-1.0,1.0
3,-1.0,1.0,1.0,1.0
4,-1.0,-1.0,-1.0,1.0


### Permutation and Random Sampling

#### 抽样的三种方法
#### 1. sampler = np.random.permutation(5)  再用 iloc或df.take(sampler) 抽取
#### 2. df.sample(n=3)，这个是不放回抽样，所以n需要比样本总数小
#### 3. 要改成放回抽样，可以设定 replace=True，就可以n比样本数大了

In [51]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([3, 0, 1, 4, 2])

In [52]:
df
df.take(sampler)

,0,1,2,3
3,12,13,14,15
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19
2,8,9,10,11


In [57]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15


In [54]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
2   -1
4    4
0    5
3    6
4    4
2   -1
2   -1
0    5
3    6
dtype: int64

### Computing Indicator/Dummy Variables

#### 把分类变量改成虚拟变量
#### 1. pd.get_dummies 可以以df的形式返回 几列0/1变量，以原来的变量值作为列名
#### 2. 如果想在列名前面加前缀 可以给 pd.get_dummies 增加参数 prefix
#### 3. 按行拼接df 用 .join

In [58]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [59]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


#### 如果需要转换的是多选题，就要手动调整一下
#### (1)建一个空list  (2)把每行的这一列循环出来，分割成每行一个list，粘入空list    (3)用 pd.unique 去除重复值
#### (4)建一个全部为0的df   (5)dummies.columns.get_indexer(每行得到的小list) 可以获得 0 1 2 这样的列序号   (6) 按序号出现的位置填入1

In [60]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

<ipython-input-60-48314dfb286d>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [61]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [62]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [63]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [71]:
gen = movies.genres[0]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

#### for i, gen in enumerate 这种代替单纯的for循环，可以多一个循环序号
#### dummies.add_prefix('Genre_')，dummies是一个df，可以直接加前缀

In [65]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [66]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                             1.0
Genre_Children's                            1.0
                               ...             
Genre_War                                   0.0
Genre_Musical                               0.0
Genre_Mystery                               0.0
Genre_Film-Noir                             0.0
Genre_Western                               0.0
Name: 0, Length: 21, dtype: object

In [67]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

#### 前面学习的按照断点分组，也可以pd.get_dummies得到各个区间从属的0/1变量df

In [68]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

### String Object Methods

#### 1. 对字符串分割并形成列表    .split
#### 2. 去除空格    .strip()
#### 3. 把字符串列表粘起来并有连接符号 '::'.join(list)
#### 4. 查找字符串中有没有：xxx  in  yyy  返回bool，.index和.find返回序号，但查找不到时.index抱错 .find报-1 
#### 5.    .count 查找字符串中包含某个字符的数目
#### 6.    .replace(xxx, yyy) 替换

In [72]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [73]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [74]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [75]:
'::'.join(pieces)

'a::b::guido'

In [77]:
'guido' in val

True

In [78]:
val.index(',')

1

In [85]:
val.find(':')

-1

In [80]:
val.index(':')

ValueError: substring not found

In [81]:
val.count(',')

2

In [82]:
val.replace(',', '::')

'a::b::  guido'

In [83]:
val.replace(',', '')

'ab  guido'

### Regular Expressions

### 正则表达式比较复杂 可以单独学习

#### re模块的函数可以分为三个大类：模式匹配、替换以及拆分。当然，它们之间是相辅相成的。一个regex描述了需要在文本中定位的一个模式，它可以用于许多目的。
#### 假设我想要拆分一个字符串，分隔符为数量不定的一组空白符（制表符、空格、换行符等）。描述一个或多个空白符的regex是\s+：

In [112]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

#### 调用re.split('\s+',text)时，正则表达式会先被编译，然后再在text上调用其split方法。你可以用re.compile自己编译regex以得到一个可重用的regex对象：

In [113]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

#### 如果只希望得到匹配regex的所有模式，则可以使用findall方法：

In [88]:
regex.findall(text)

['    ', '\t ', '  \t']

#### 如果打算对许多字符串应用同一条正则表达式，强烈建议通过re.compile创建regex对象。这样将可以节省大量的CPU时间。

#### match和search跟findall功能类似。findall返回的是字符串中所有的匹配项，而search则只返回第一个匹配项。match更加严格，它只匹配字符串的首部。
#### 假设我们有一段文本以及一条能够识别大部分电子邮件地址的正则表达式：

In [89]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

#### 对text使用findall将得到一组电子邮件地址：

In [90]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

#### search返回的是文本中第一个电子邮件地址（以特殊的匹配项对象形式返回）。对于上面那个regex，匹配项对象只能告诉我们模式在原字符串中的起始和结束位置：

In [91]:
m = regex.search(text)
m
text[m.start():m.end()]

'dave@google.com'

#### regex.match则将返回None，因为它只匹配出现在字符串开头的模式：

In [92]:
print(regex.match(text))

None


#### 相关的，sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的新字符串：

In [93]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



#### 假设你不仅想要找出电子邮件地址，还想将各个地址分成3个部分：用户名、域名以及域后缀。要实现此功能，只需将待分段的模式的各部分用圆括号包起来即可：

In [94]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

#### 由这种修改过的正则表达式所产生的匹配项对象，可以通过其groups方法返回一个由模式各段组成的元组：

In [95]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

#### 对于带有分组功能的模式，findall会返回一个元组列表：

In [96]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

#### sub还能通过诸如\1、\2之类的特殊符号访问各匹配项中的分组。符号\1对应第一个匹配的组，\2对应第二个匹配的组，以此类推：

In [97]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas

In [115]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

#### 通过data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。
#### 为了解决这个问题，Series有一些能够跳过NA值的面向数组方法，进行字符串操作。通过Series的str属性即可访问这些方法。
#### 例如，我们可以通过str.contains检查各个电子邮件地址是否含有"gmail"：

In [103]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

#### 也可以使用正则表达式，还可以加上任意re选项（如IGNORECASE）：

In [105]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

#### 有两个办法可以实现矢量化的元素获取操作：要么使用str.get，要么在str属性上使用索引：

In [106]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

#### 要访问嵌入列表中的元素，我们可以传递索引到这两个函数中：

In [109]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

#### 你可以利用这种方法对字符串进行截取：

In [110]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion